In [1]:
import torch
import esm

In [2]:
eval_model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
eval_model = eval_model.to("cuda")
eval_model = eval_model.eval()
alphabet.all_toks

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t12_35M_UR50D.pt" to /home/kkj/.cache/torch/hub/checkpoints/esm2_t12_35M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t12_35M_UR50D-contact-regression.pt" to /home/kkj/.cache/torch/hub/checkpoints/esm2_t12_35M_UR50D-contact-regression.pt


['<cls>',
 '<pad>',
 '<eos>',
 '<unk>',
 'L',
 'A',
 'G',
 'V',
 'S',
 'E',
 'R',
 'T',
 'I',
 'D',
 'P',
 'K',
 'Q',
 'N',
 'F',
 'Y',
 'M',
 'H',
 'W',
 'C',
 'X',
 'B',
 'U',
 'Z',
 'O',
 '.',
 '-',
 '<null_1>',
 '<mask>']

In [16]:
batch_converter = alphabet.get_batch_converter()

data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGI"),
]

batch_labels, batch_strs, batch_tokens = batch_converter(data)
print(batch_tokens.shape)
print(batch_labels)
print(batch_strs)
print(batch_tokens)
print(alphabet.encode("<cls>MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"))

torch.Size([2, 67])
['protein1', 'protein2']
['MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG', 'KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGI']
tensor([[ 0, 20, 15, 11,  7, 10, 16,  9, 10,  4, 15,  8, 12,  7, 10, 12,  4,  9,
         10,  8, 15,  9, 14,  7,  8,  6,  5, 16,  4,  5,  9,  9,  4,  8,  7,  8,
         10, 16,  7, 12,  7, 16, 13, 12,  5, 19,  4, 10,  8,  4,  6, 19, 17, 12,
          7,  5, 11, 14, 10,  6, 19,  7,  4,  5,  6,  6,  2],
        [ 0, 15,  5,  4, 11,  5, 10, 16, 16,  9,  7, 18, 13,  4, 12, 10, 13, 21,
         12,  8, 16, 11,  6, 20, 14, 14, 11, 10,  5,  9, 12,  5, 16, 10,  4,  6,
         18, 10,  8, 14, 17,  5,  5,  9,  9, 21,  4, 15,  5,  4,  5, 10, 15,  6,
          7, 12,  9, 12,  7,  8,  6,  5,  8, 10,  6, 12,  2]])
[0, 20, 15, 11, 7, 10, 16, 9, 10, 4, 15, 8, 12, 7, 10, 12, 4, 9, 10, 8, 15, 9, 14, 7, 8, 6, 5, 16, 4, 5, 9, 9, 4, 8, 7, 8, 10, 16, 7, 12, 7, 16, 13, 12, 5, 19, 4, 10, 8, 4, 6, 19, 17, 12, 7, 5, 11, 14, 10,

In [4]:
## The absorb mapping
# "A": 0,
# "C": 1,
# "D": 2,
# "E": 3,
# "F": 4,
# "G": 5,
# "H": 6,
# "I": 7,
# "K": 8,
# "L": 9,
# "M": 10,
# "N": 11,
# "P": 12,
# "Q": 13,
# "R": 14,
# "S": 15,
# "T": 16,
# "V": 17,
# "W": 18,
# "Y": 19,
# "?": 20,
# "[": 21,
# "]": 22,
# "-": 23

## The esm mapping
# ['<cls>',
#  '<pad>',
#  '<eos>',
#  '<unk>',
#  'L',
#  'A',
#  'G',
#  'V',
#  'S',
#  'E',
#  'R',
#  'T',
#  'I',
#  'D',
#  'P',
#  'K',
#  'Q',
#  'N',
#  'F',
#  'Y',
#  'M',
#  'H',
#  'W',
#  'C',
#  'X',
#  'B',
#  'U',
#  'Z',
#  'O',
#  '.',
#  '-',
#  '<null_1>',
#  '<mask>']

from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/axolotl/tokenizer/tokenizer_absorb")

esm_token_mapping = [[] for _ in range(tokenizer.vocab_size)]

for i, tok in enumerate(alphabet.all_toks):
    if tok == '<cls>':
        in_tok = '['
    elif tok == '<eos>':
        in_tok = ']'
    elif tok == '<unk>':
        in_tok = '?'
    elif tok == '<mask>':
        in_tok = '-'
    elif tok == '-': # '-' is what i use as mask token
        in_tok = '?'
    else:
        in_tok = tok

    tok_id = tokenizer.encode(in_tok)[1]
    print(tok, tok_id)
    esm_token_mapping[tok_id].append(i)

print(esm_token_mapping)

<cls> 21
<pad> 20
<eos> 22
<unk> 20
L 9
A 0
G 5
V 17
S 15
E 3
R 14
T 16
I 7
D 2
P 12
K 8
Q 13
N 11
F 4
Y 19
M 10
H 6
W 18
C 1
X 20
B 20
U 20
Z 20
O 20
. 20
- 20
<null_1> 20
<mask> 23
[[5], [23], [13], [9], [18], [6], [21], [12], [15], [4], [20], [17], [14], [16], [10], [8], [11], [7], [22], [19], [1, 3, 24, 25, 26, 27, 28, 29, 30, 31], [0], [2], [32]]


/home/kkj/axolotl/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_logits = torch.randn(2, 67, 23)
esm_logits = torch.zeros(2, 67, 33)

batch_size = batch_tokens.shape[0]
for i in range(batch_tokens.shape[-1]):
    batch_tokens_masked = batch_tokens.clone()
    batch_tokens_masked[:, i] = alphabet.mask_idx
    batch_tokens_masked = batch_tokens_masked.to("cuda")
    with torch.no_grad():
        logits = eval_model(batch_tokens_masked)["logits"]
    esm_logits[:, i, :] = logits[:, i, :]

new_esm_logits = torch.zeros(2, 67, 23)
for i in range(model_logits.shape[-1]):
    new_esm_logits[:, :, i] = esm_logits[:, :, esm_token_mapping[i]].sum(dim=-1)

33


NameError: name 'eval_model' is not defined

In [6]:
import torch.nn.functional as F

perplexity = F.cross_entropy(new_esm_logits, model_logits, reduction="none").mean(dim=-1).exp().mean()
print(perplexity)

del eval_model, logits

tensor(120.4476)
